In [96]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [98]:
dfuse=pd.read_csv("dfuse.csv")
dfuse.head()

,tweetID,text,hashtags,urls,favorite_count,retweet_count
0,665775631632732160,RT @GOP: Hillary refuses to say we are at war ...,DemDebate,NaN,0,335
1,665775625974603776,"As campaign focus turns to foreign policy, Cli...",NaN,https://t.co/zZPfrjntpN,1,0
2,665775625660166145,"RT @realDonaldTrump: ""@redletter99: @realDonal...",NaN,NaN,0,867
3,665775615451205632,RT @MMFlint: Hillary talks about Wall Street p...,DemDebate,NaN,0,731
4,665775614486556672,"RT @HillaryClinton: ""President Obama deserves ...",DemDebate,NaN,0,2454


In [99]:
practice = dfuse.text[1]
practice

'As campaign focus turns to foreign policy, Clinton points to her experience - Washington Post https://t.co/zZPfrjntpN'

In [100]:
import labMTsimple.storyLab as sl

In [101]:
lang = 'english'
labMT,labMTvector,labMTwordList = sl.emotionFileReader(stopval=0.0,lang=lang,returnVector=True)

In [102]:
practiceValence,practiceFvec = sl.emotion(practice,labMT,shift=True,happsList=labMTvector)
practiceValence
practiceStoppedVec = sl.stopper(practiceFvec,labMTvector,labMTwordList,stopVal=1.0)
finalValence = sl.emotionV(practiceStoppedVec,labMTvector)

In [103]:
for i in range(0, len(practiceFvec)):
    if practiceFvec[i] == 1:
        print labMTwordList[i]

experience
focus
points
washington
her
clinton
foreign
post
campaign
as
turns
policy


In [104]:
varlisttest = list()
for i in range(0, len(practiceStoppedVec)):
    if practiceStoppedVec[i] == 1:
        varlisttest.append(labMTvector[i])

In [105]:
for i in range(0, len(practiceStoppedVec)):
    if practiceStoppedVec[i] == 1:
        print labMTwordList[i]

experience
focus
points
washington


In [106]:
import labMTsimple.storyLab as sl

# importing the dictionary
lang = 'english'
labMT,labMTvector,labMTwordList = sl.emotionFileReader(stopval=0.0,lang=lang,returnVector=True)

# dfuse['sentiment'] = 0
# dfuse['sentiment_var'] = 0

sentiment_list = list()
variance_list = list()

for i in range(0, len(dfuse)):
    tweettext = dfuse.text[i]
    # calculate sentiment score for each tweet
    tweetValence,tweetFvec = sl.emotion(tweettext,labMT,shift=True,happsList=labMTvector)
    
    # set score for neutral words to 0 and generate real sentiment score
    tweetStoppedVec = sl.stopper(tweetFvec,labMTvector,labMTwordList,stopVal=1.0)
    finaltweetsent = sl.emotionV(tweetStoppedVec,labMTvector)
    
    # handle corner case for when all words are neutral. emotionV automatically sets it to -1 when words are all neutral
    # we will set to 5 because it is in the middle of our 1 to 9 happiness scale
    if finaltweetsent == -1:
        finaltweetsent = 5
    
    # calculate variance of sentiment of words in each tweet
    varlist = list()
    for i in range(0, len(tweetStoppedVec)):
        if tweetStoppedVec[i] == 1:
            varlist.append(labMTvector[i])
    tweetvariance = np.var(varlist)
    # handle corner case for tweet with no sentiment score
    if np.isnan(tweetvariance):
        tweetvariance = 0
    
    sentiment_list.append(finaltweetsent)
    variance_list.append(tweetvariance)    
    
# store values in dataframe
dfuse['sentiment'] = sentiment_list
dfuse['sentiment_var'] = variance_list
    
    
    
    
    

In [108]:
dfuse

,tweetID,text,hashtags,urls,favorite_count,retweet_count,sentiment,sentiment_var
0,665775631632732160,RT @GOP: Hillary refuses to say we are at war ...,DemDebate,NaN,0,335,4.090000,5.244100
1,665775625974603776,"As campaign focus turns to foreign policy, Cli...",NaN,https://t.co/zZPfrjntpN,1,0,6.135000,0.015075
2,665775625660166145,"RT @realDonaldTrump: ""@redletter99: @realDonal...",NaN,NaN,0,867,5.470000,4.146700
3,665775615451205632,RT @MMFlint: Hillary talks about Wall Street p...,DemDebate,NaN,0,731,5.280000,4.531467
4,665775614486556672,"RT @HillaryClinton: ""President Obama deserves ...",DemDebate,NaN,0,2454,6.390000,0.022500
5,665775612733227008,Hillary Clinton's 10 Most Hawkish Moments From...,NaN,https://t.co/L3sCKaX34Q,0,0,6.213333,0.008089
6,665775608941699073,RT @gov: Top Tweeted moment of the #DemDebate:...,DemDebate,https://t.co/8WaWqLBoK1,0,141,6.750000,0.000100
7,665775608123826177,Watch The Full Second Democratic Debate Here -...,NaN,https://t.co/AATmYNZrfy,0,0,6.500000,0.320000
8,665775604709609472,"RT @NationalMemo: Hillary: ""I think President ...",DemDebate,NaN,0,228,6.326667,0.023022
9,665775603979853824,RT @thedailybeast: Hillary Escapes Another #De...,DemDebate,https://t.co/DMvcjAYDym,0,8,5.000000,0.000000
